In [7]:
# Imports and set up

# Given the self-contained design of VBT, a single import is enough for the analysis.
import vectorbt as vbt
import yfinance as yahooFinance
vbt.settings.set_theme("dark")

In [12]:
SYMBOL = "BTC-USD"
TIMEFRAME = "60m"
START = "2023-01-01"
END = "2023-11-11"

LAST_N_BARS = 66
PRED_N_BARS = 12

GIF_FNAME = "projections.gif"
GIF_N_BARS = 72
GIF_FPS = 4
GIF_PAD = 0.01

# print(vbt.YFData.get_available_symbols())

data = yahooFinance.download(SYMBOL, start=START, interval=TIMEFRAME)
data

# data = vbt.YFData.pull(SYMBOL, timeframe=TIMEFRAME, start=START)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-01-01 00:00:00,16547.914062,16553.699219,16529.074219,16539.396484,16539.396484,0
2023-01-01 01:00:00,16538.013672,16556.140625,16536.416016,16556.140625,16556.140625,0
2023-01-01 02:00:00,16556.494141,16564.591797,16549.503906,16557.580078,16557.580078,0
2023-01-01 03:00:00,16557.521484,16557.912109,16540.302734,16548.863281,16548.863281,0
2023-01-01 04:00:00,16546.429688,16548.511719,16528.251953,16530.708984,16530.708984,16758784
...,...,...,...,...,...,...
2023-11-12 04:00:00,36988.152344,37027.531250,36938.621094,37005.789062,37005.789062,0
2023-11-12 05:00:00,37007.386719,37123.867188,36960.800781,37123.867188,37123.867188,0
2023-11-12 06:00:00,37120.875000,37170.582031,37096.476562,37165.968750,37165.968750,33020928


In [14]:
def find_patterns(data):
    price = data.Close
    pattern = price.values[-LAST_N_BARS:]
    pattern_ranges = price.vbt.find_pattern(
        pattern=pattern,
        rescale_mode="rebase",
        overlap_mode="allow",
        wrapper_kwargs=dict(freq=TIMEFRAME)
    )
    pattern_ranges = pattern_ranges.status_closed
    return pattern_ranges

pattern_ranges = find_patterns(data)

AttributeError: 'Vbt_SRAccessor' object has no attribute 'find_pattern'

In [ ]:
def plot_projections(data, pattern_ranges, **kwargs):
    projection_ranges = pattern_ranges.with_delta(
        PRED_N_BARS,
        open=data.open,
        high=data.high,
        low=data.low,
        close=data.close,
    )
    projection_ranges = projection_ranges.status_closed
    return projection_ranges.plot_projections(
        plot_past_period=LAST_N_BARS, 
        **kwargs,
    )

plot_projections(data, pattern_ranges, plot_bands=True).show_png()

In [ ]:

def plot_frame(frame_index, **kwargs):
    sub_data = data.loc[:frame_index[-1]]
    pattern_ranges = find_patterns(sub_data)
    if pattern_ranges.count() < 3:
        return None
    return plot_projections(sub_data, pattern_ranges, **kwargs)

vbt.save_animation(
    GIF_FNAME,
    data.index[-GIF_N_BARS:],
    plot_frame,
    plot_projections=False,
    delta=1,
    fps=GIF_FPS,
    writer_kwargs=dict(loop=0),
    yaxis_range=[
        data.low.iloc[-GIF_N_BARS:].min() * (1 - GIF_PAD), 
        data.high.iloc[-GIF_N_BARS:].max() * (1 + GIF_PAD)
    ],
)